In [161]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import matplotlib.pyplot as plt
# import seaborn
import re

import numpy
import json 
import numpy as np

Get Each Universities Link

In [162]:
regions_name = ['America & Canada', 'Asia', 'Australia', 'Europe', 'UK & Ireland']
main_page = requests.get("https://iisma.kemdikbud.go.id/info/host-universities-list/")
soup = BeautifulSoup(main_page.text, features='html.parser')
regions = soup.find_all('div', class_='elementor-image-carousel swiper-wrapper')
links_by_region = {}
for i, region in enumerate(regions):
    links = []
    universities = region.find('a')
    link = universities['href']
    links = link.split(',')
    links_by_region[regions_name[i]] = links

In [163]:
links_by_region

{'America & Canada': ['https://iisma.kemdikbud.go.id/info/31-boston-university-metropolitan-college/',
  'https://iisma.kemdikbud.go.id/info/56-arizona-state-university/',
  'https://iisma.kemdikbud.go.id/info/05-the-university-of-pennsylvania-college-of-liberal-and-professional-studies/',
  'https://iisma.kemdikbud.go.id/info/06-yale-university/',
  'https://iisma.kemdikbud.go.id/info/29-penn-state-university',
  'https://iisma.kemdikbud.go.id/info/30-university-of-california-davis/',
  'https://iisma.kemdikbud.go.id/info/35-uc-chile/',
  'https://iisma.kemdikbud.go.id/info/45-michigan-state-university/',
  'https://iisma.kemdikbud.go.id/info/64-university-of-colorado-boulder/',
  'https://iisma.kemdikbud.go.id/info/13-university-of-british-columbia/',
  'https://iisma.kemdikbud.go.id/info/42-university-of-waterloo/',
  'https://iisma.kemdikbud.go.id/info/46-western-university/',
  'https://iisma.kemdikbud.go.id/info/03-university-of-chicago/',
  'https://iisma.kemdikbud.go.id/info/21

Get Universities Primary Information

In [164]:
column_name = ["Name", "Image", "Location", "Description", "Official Link", "IISMA Link"]
df = pd.DataFrame(columns=column_name)

for region in links_by_region:
    for link in links_by_region[region]:
        details_page = requests.get(link)
        soup = BeautifulSoup(details_page.text, 'html.parser')
        try:
            # Left side
            left_info = soup.findAll("div", class_="elementor-widget-container")
            image = left_info[0].img['src']
            name = left_info[1].h4.text
            location = left_info[2].p.text
            official_link = left_info[4].text.strip()
            
            # Right side
            right_info = soup.findAll("div", class_="elementor-tab-content elementor-clearfix")
            description = right_info[0].text
            
            # Make dataframe
            new_row = [name, image, location, description, official_link, link]
        except:
            new_row = ["", "", "", "", "", link]
            print(link)

        new_row_df = pd.DataFrame([new_row], columns=column_name)
        df = pd.concat([df, new_row_df], ignore_index = True)


In [165]:
region_list = []
for region in links_by_region.keys():
    for i in links_by_region[region]:
        region_list.append(region)
df["Region"] = region_list

In [166]:
df

,Name,Image,Location,Description,Official Link,IISMA Link,Region
0,Boston University,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,"Boston, Massachusetts, USA",the MET International Undergraduate Program pr...,bu.edu/metinternational,https://iisma.kemdikbud.go.id/info/31-boston-u...,America & Canada
1,Arizona State University,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,"Tempe, Arizona, USA","Arizona State University (ASU), located in Tem...",www.asu.edu,https://iisma.kemdikbud.go.id/info/56-arizona-...,America & Canada
2,The University of Pennsylvania,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,United States of America,Penn is a private urban research university wi...,www.lps.upenn.edu/non-degree-programs/igsp,https://iisma.kemdikbud.go.id/info/05-the-univ...,America & Canada
3,Yale University,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,United States of America,"Since its founding in 1701, Yale has been dedi...",https://summer.yale.edu/,https://iisma.kemdikbud.go.id/info/06-yale-uni...,America & Canada
4,Penn State University,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,United States of America,"Penn State University, founded in 1855, is the...",global.psu.edu,https://iisma.kemdikbud.go.id/info/29-penn-sta...,America & Canada
...,...,...,...,...,...,...,...
107,University of Bath,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,United Kingdom,The University of Bath received its Royal Char...,www.ucl.ac.uk,https://iisma.kemdikbud.go.id/info/s93-univers...,UK & Ireland
108,University of Limerick,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,Ireland,"Established in 1972, the University of Limeric...",www.ucl.ac.uk,https://iisma.kemdikbud.go.id/info/s96-univers...,UK & Ireland
109,University of Southampton,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,"Southampton, United Kingdom",University of Southampton (UoS) is a research ...,southampton.ac.uk,https://iisma.kemdikbud.go.id/info/s91-univers...,UK & Ireland
110,Queen's University Belfast,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,United Kingdom,Queen’s University has been making a differenc...,www.qub.ac.uk,https://iisma.kemdikbud.go.id/info/s94-queens-...,UK & Ireland


Get Universities Requirements

In [167]:
def get_number(text, toFloat: bool):
    match = re.search(r'\b\d+(?:\.\d+)?\b', text)

    if match:
        if toFloat:
            value = str(float(match.group()))
        else:
            value = str(match.group())
    else:
        value = "-"

    return value

In [168]:
requirements_data = []
browser_options = webdriver.ChromeOptions()
browser_options.add_argument('--headless')
browser_options.headless = True
driver = webdriver.Chrome(options=browser_options)

for region in links_by_region:
    for link in links_by_region[region]:
        data = {}
        driver.get(link)

        try:
            driver.find_element(By.ID, "elementor-tab-title-4502").click()
            requirements = driver.find_element(By.ID,'elementor-tab-content-4502').text.split("\n")
        except NoSuchElementException:
            try:
                driver.find_element(By.ID, "elementor-tab-title-1812").click()
                requirements = driver.find_element(By.ID,'elementor-tab-title-1812').text.split("\n")
            except NoSuchElementException:
                print(link)
                data["GPA"] = ""
                data["IELTS"] = ""
                data["DET"] = ""
                data["TOEFL"] = ""
                requirements_data.append(data)
                continue
        
        for req in requirements:
            if "GPA" in req:
                data["GPA"] = get_number(req, True)
            elif "English Language" not in req:
                if "IELTS" in req:
                    data["IELTS"] = get_number(req, True)
                elif "DET" in req or "Duolingo" in req:
                    data["DET"] = get_number(req, False)
                elif "TOEFL" in req:
                    data["TOEFL"] = get_number(req, False)
        requirements_data.append(data)

driver.quit()

In [169]:
df_requirements = pd.DataFrame(requirements_data)
df_requirements["University"] = df["Name"]
df_requirements

,GPA,IELTS,TOEFL,DET,University
0,3.0,6.0,84,115,Boston University
1,3.0,6.0,79,115,Arizona State University
2,3.3,7.0,100,120,The University of Pennsylvania
3,3.0,7.0,100,120,Yale University
4,3.0,6.5,80,120,Penn State University
...,...,...,...,...,...
107,3.0,6.5,90,120,University of Bath
108,3.0,6.0,78,100,University of Limerick
109,3.0,6.0,92,125,University of Southampton
110,3.0,6.5,90,-,Queen's University Belfast


Get Universities Requirements

In [170]:
def get_range(text):
    parts = text.split(":")
    # match = re.search(r'\b\d+(?:\.\d+)?\b', text)

    if len(parts) > 1:
        range = parts[1].strip()
        return range

Opsi 1

In [171]:
intakes_data = []
browser_options = webdriver.ChromeOptions()
browser_options.add_argument('--headless')
browser_options.headless = True
driver = webdriver.Chrome(options=browser_options)

# for x in range(1):
for region in links_by_region:
    # for link in links_by_region["America & Canada"][:1]:
    for link in links_by_region[region]:
        data = {}
        driver.get(link)

        try:
            tab_click = driver.find_element(By.ID, "elementor-tab-title-4504")
            tab_click.click()
            intakes = driver.find_element(By.ID,'elementor-tab-content-4504')
        except NoSuchElementException:
            try:
                tab_click = driver.find_element(By.ID, "elementor-tab-title-1814")
                tab_click.click()
                intakes = driver.find_element(By.ID,'elementor-tab-title-1814')
            except NoSuchElementException:
                # print(link)
                # data["GPA Min"] = ""
                # data["GPA Max"] = ""
                # data["IELTS Min"] = ""
                # data["IELTS Max"] = ""
                # data["DET Min"] = ""
                # data["DET Max"] = ""
                # data["TOEFL Min"] = ""
                # data["TOEFL Max"] = ""
                intakes_data.append(data)
                continue

        intakes_list = intakes.find_elements(By.TAG_NAME, "p")
        for intake in intakes_list:
            data = {}
            data["University"] = driver.find_element(By.TAG_NAME,"h4").text
            splitted_intakes = intake.text.split("\n") 
            for intake in splitted_intakes:
                if len(intake) > 0:
                    if ":" not in intake: 
                        if "Applicants" in intake:
                            key1 = "Applicants"
                        elif "Awardees" in intake:
                            key1 = "Awardees"

                        try:
                            year = re.findall(r"\d{4}", intake)[0]
                        except:
                            year = re.findall(r"\d{4}", tab_click.text)[0] # Kalau gak ada tahunnya, ikut judul tab
                    else:
                        data["Year"] = year
                        if "student" in intake:
                            if "Cofunding" in intake or "Co-funding" in intake:
                                key = "Co-funding"
                            else:
                                key = "Regular"
                            data[f"{key} {key1}"] = get_number(intake, False)
                        else:
                            key2 = ""
                            if "GPA" in intake:
                                key2 = "GPA"
                            elif "IELTS" in intake:
                                key2 = "IELTS"
                            elif "DET" in intake or "Duolingo" in intake:
                                key2 = "DET"
                            elif "TOEFL" in intake:
                                key2 = "TOEFL"

                            ranges = get_range(intake)
                            value = "–"
                            if len(ranges) > 1:
                                if "-" in ranges:
                                    value = ranges.split("-")
                                elif "–" in ranges:
                                    value = ranges.split("–")

                            try:
                                data[f"{key} {key1} {key2} Min"] = value[0]
                                data[f"{key} {key1} {key2} Max"] = value[1]
                            except IndexError:
                                data[f"{key} {key1} {key2} Min"] = value[0]
                                data[f"{key} {key1} {key2} Max"] = value[0]

            # print(link)
            # print(splitted_intakes)
            intakes_data.append(data)

driver.quit()

In [172]:
# print(intakes_data)
df_intakes = pd.DataFrame(intakes_data).groupby(["University", "Year"]).sum().reset_index()
display(df_intakes.iloc[:, 15:26])
# df_intakes["University"] = df["Name"]
# display(df_intakes.loc[df_intakes["University"] == "Vytautas Magnus University"].iloc[:, :19])
# display(df_intakes.iloc[:, :19])
# display(df_intakes.iloc[:, 10:20])
# display(df_intakes["University"])

,Regular Awardees TOEFL Max,Regular Awardees IELTS Min,Regular Awardees IELTS Max,Regular Awardees DET Min,Regular Awardees DET Max,Co-funding Applicants,Co-funding Applicants GPA Min,Co-funding Applicants GPA Max,Co-funding Applicants TOEFL Min,Co-funding Applicants TOEFL Max,Co-funding Applicants IELTS Min
0,–,–,–,140,145,0,0,0,0,0,0
1,–,7.5,7.5,130,145,0,0,0,0,0,0
2,–,–,–,140,155,0,0,0,0,0,0
3,–,8.5,8.5,140,155,9,3.31,3.81,–,–,6.5
4,–,–,–,135,145,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
121,–,–,–,105,130,0,0,0,0,0,0
122,–,–,–,125,140,55,3.21,3.93,–,–,6
123,–,–,–,140,145,0,0,0,0,0,0
124,–,7.5,8,130,160,0,0,0,0,0,0


Opsi 2

In [173]:
intakes_data = []
browser_options = webdriver.ChromeOptions()
browser_options.add_argument('--headless')
browser_options.headless = True
driver = webdriver.Chrome(options=browser_options)

# for x in range(1):
for region in links_by_region:
    # for link in links_by_region["America & Canada"][:1]:
    for link in links_by_region[region]:
        data = {}
        driver.get(link)

        try:
            tab_click = driver.find_element(By.ID, "elementor-tab-title-4504")
            tab_click.click()
            intakes = driver.find_element(By.ID,'elementor-tab-content-4504')
        except NoSuchElementException:
            try:
                tab_click = driver.find_element(By.ID, "elementor-tab-title-1814")
                tab_click.click()
                intakes = driver.find_element(By.ID,'elementor-tab-title-1814')
            except NoSuchElementException:
                # print(link)
                # data["GPA Min"] = ""
                # data["GPA Max"] = ""
                # data["IELTS Min"] = ""
                # data["IELTS Max"] = ""
                # data["DET Min"] = ""
                # data["DET Max"] = ""
                # data["TOEFL Min"] = ""
                # data["TOEFL Max"] = ""
                intakes_data.append(data)
                continue

        intakes_list = intakes.find_elements(By.TAG_NAME, "p")
        for intake in intakes_list:
            data = {}
            data["University"] = driver.find_element(By.TAG_NAME,"h4").text
            splitted_intakes = intake.text.split("\n") 
            for intake in splitted_intakes:
                if len(intake) > 0:
                    if ":" not in intake: 
                        if "Applicants" in intake:
                            key = "Applicants"
                        elif "Awardees" in intake:
                            key = "Awardees"

                        try:
                            year = re.findall(r"\d{4}", intake)[0]
                        except:
                            year = re.findall(r"\d{4}", tab_click.text)[0] # Kalau gak ada tahunnya, ikut judul tab
                    else:
                        if "Cofunding" in intake or "Co-funding" in intake:
                            data["Intake"] = "Co-funding Intake " + year
                        elif "Regular" in intake:
                            data["Intake"] = "Regular Intake " + year
                        else:
                            if "students" in intake:
                                data["Intake"] = "Regular Intake " + year
                        
                        if "student" in intake:
                            data[key] = get_number(intake, False)
                        else:
                            key2 = ""
                            if "GPA" in intake:
                                key2 = "GPA"
                            elif "IELTS" in intake:
                                key2 = "IELTS"
                            elif "DET" in intake or "Duolingo" in intake:
                                key2 = "DET"
                            elif "TOEFL" in intake:
                                key2 = "TOEFL"

                            ranges = get_range(intake)
                            value = "–"
                            if len(ranges) > 1:
                                if "-" in ranges:
                                    value = ranges.split("-")
                                elif "–" in ranges:
                                    value = ranges.split("–")

                            if len(key2) > 1:
                                try:
                                    data[f"{key} {key2} Min"] = value[0]
                                    data[f"{key} {key2} Max"] = value[1]
                                except IndexError:
                                    data[f"{key} {key2} Min"] = value[0]
                                    data[f"{key} {key2} Max"] = value[0]

            # print(link)
            # print(splitted_intakes)
            intakes_data.append(data)

driver.quit()

In [174]:
# print(intakes_data)
df_intakes = pd.DataFrame(intakes_data)
display(df_intakes.groupby(["University", "Intake"]).sum().iloc[130:137, :])
# df_intakes["University"] = df["Name"]
# display(df_intakes.loc[df_intakes["University"] == "Vytautas Magnus University"].iloc[:, :19])
# display(df_intakes.iloc[:, :19])
# display(df_intakes.iloc[:, 10:20])
# display(df_intakes["University"])

Applicants  \
University                        Intake                              
University of Zagreb              Regular Intake 2023           214   
Victoria University of Wellington Regular Intake 2023           258   
Vrije Universiteit Amsterdam      Regular Intake 2022           174   
                                  Regular Intake 2023            62   
Vytautas Magnus University        Co-funding Intake 2023         55   
                                  Regular Intake 2022           241   
                                  Regular Intake 2023           463   

                                                         Applicants GPA Min  \
University                        Intake                                      
University of Zagreb              Regular Intake 2023                  3.03   
Victoria University of Wellington Regular Intake 2023                  2.88   
Vrije Universiteit Amsterdam      Regular Intake 2022                  3.05   
                                  Regular Intake 2023                  3.19   
Vytautas Magnus University        Co-funding Intake 2023               3.21   
                                  Regular Intake 2022                  3.04   
                                  Regular Intake 2023                  3.08   

                                                         Applicants GPA Max  \
University                        Intake                                      
University of Zagreb              Regular Intake 2023                  3.99   
Victoria University of Wellington Regular Intake 2023                  4.00   
Vrije Universiteit Amsterdam      Regular Intake 2022                  3.98   
                                  Regular Intake 2023                  3.97   
Vytautas Magnus University        Co-funding Intake 2023               3.93   
                                  Regular Intake 2022                  3.99   
                                  Regular Intake 2023                  4.00   

                                                         Applicants TOEFL Min  \
University                        Intake                                        
University of Zagreb              Regular Intake 2023                       –   
Victoria University of Wellington Regular Intake 2023                      85   
Vrije Universiteit Amsterdam      Regular Intake 2022                      96   
                                  Regular Intake 2023                      99   
Vytautas Magnus University        Co-funding Intake 2023                    –   
                                  Regular Intake 2022                       –   
                                  Regular Intake 2023                       –   

                                                         Applicants TOEFL Max  \
University                        Intake                                        
University of Zagreb              Regular Intake 2023                       –   
Victoria University of Wellington Regular Intake 2023                      99   
Vrije Universiteit Amsterdam      Regular Intake 2022                     106   
                                  Regular Intake 2023                     115   
Vytautas Magnus University        Co-funding Intake 2023                    –   
                                  Regular Intake 2022                       –   
                                  Regular Intake 2023                       –   

                                                         Applicants IELTS Min  \
University                        Intake                                        
University of Zagreb              Regular Intake 2023                       6   
Victoria University of Wellington Regular Intake 2023                       6   
Vrije Universiteit Amsterdam      Regular Intake 2022                     6.5   
                                  Regular Intake 2023                     6.5   
Vytautas Magnus University        Co-funding In